<img style="float: left;" src="http://third.datacastle.cn/pkbigdata/master.other.img/8ef429f9-2032-47b8-bcc4-818fa9e41a25.png" width="50%">

In [1]:
import numpy as np
import pandas as pd
import time

In [2]:
op_train = pd.read_csv('../data/operation_train_new.csv')
tran_train = pd.read_csv('../data/transaction_train_new.csv')
tag_train = pd.read_csv('../data/tag_train_new.csv')

In [3]:
print(op_train.shape)
print(tran_train.shape)

(1460843, 20)
(264654, 27)


### 数据预处理

In [4]:
def operation_len(idx):
    return len(operation_train[operation_train['UID'] == idx])

def transaction_len(idx):
    return len(transaction_train[transaction_train['UID'] == idx])

def find_different_uid(op_uids, tran_uids):
    op_diff_uids = []
    for uid in op_uids:
        if uid not in tran_uids:
            op_diff_uids.append(uid)
        else:
            tran_uids.remove(uid)
#             np.delete(tran_uids, np.where(tran_uids == uid)[0])
    return op_diff_uids, tran_uids

def find_same_uid(src1_uids, src2_uids):
    same_uids = [uid for uid in src1_uids if uid in src2_uids]
    return same_uids

In [5]:
# 处理时间字符串
op_train['time'] = op_train['day'].apply(lambda x: "2018-08-%02d" % x) + ' ' + op_train['time']
op_train['timestamp'] = op_train['time'].apply(lambda x:time.mktime(time.strptime(x,'%Y-%m-%d %H:%M:%S')))

tran_train['time'] = tran_train['day'].apply(lambda x: "2018-08-%02d" % x) + ' ' + tran_train['time']
tran_train['timestamp'] = tran_train['time'].apply(lambda x:time.mktime(time.strptime(x,'%Y-%m-%d %H:%M:%S')))

In [8]:
# 根据UID进行分组，再根据timestamp进行排序
op_train = op_train.sort_values(by='UID',ascending=True)
tran_train = tran_train.sort_values(by='UID',ascending=True)
tag_train = tag_train.sort_values(by='UID', ascending=True)

op_train_gb = op_train.groupby('UID').apply(lambda x:x.sort_values(by='timestamp', ascending=True))
tran_train_gb = tran_train.groupby('UID').apply(lambda x:x.sort_values(by='timestamp', ascending=True))
# tag_train_gb = tag_train.groupby('UID', as_index=True)

In [12]:
op_train_gb.head(10)

UID  day              mode  success                 time   os  \
UID                                                                             
10000 307385  10000   13  c8741ce15ceac2a4      1.0  2018-08-13 14:20:56  103   
      420183  10000   13  c8741ce15ceac2a4      1.0  2018-08-13 14:20:56  103   
      87507   10000   13  c8741ce15ceac2a4      1.0  2018-08-13 14:20:56  103   
      216744  10000   13  270ca3ca8f0d1126      1.0  2018-08-13 14:22:59  103   
      96940   10000   26  c8741ce15ceac2a4      1.0  2018-08-26 12:23:48  103   
      196361  10000   26  c8741ce15ceac2a4      1.0  2018-08-26 12:23:48  103   
      168206  10000   26  c8741ce15ceac2a4      1.0  2018-08-26 12:23:48  103   
      339411  10000   26  d25caee90b27fa9b      1.0  2018-08-26 12:24:01  103   
      150480  10000   26  d25caee90b27fa9b      1.0  2018-08-26 12:24:21  103   
10001 214008  10001    2  c8741ce15ceac2a4      1.0  2018-08-02 08:30:52  102   

             version           device1          device2      device_code1  \
UID                                                                         
10000 307385   7.0.5  aca4977fbe8741e1         IPHONE 5               NaN   
      420183   7.0.5  aca4977fbe8741e1         IPHONE 5               NaN   
      87507    7.0.5  aca4977fbe8741e1         IPHONE 5               NaN   
      216744   7.0.5  aca4977fbe8741e1         IPHONE 5               NaN   
      96940    7.0.7  aca4977fbe8741e1         IPHONE 5               NaN   
      196361   7.0.7  aca4977fbe8741e1         IPHONE 5               NaN   
      168206   7.0.7  aca4977fbe8741e1         IPHONE 5               NaN   
      339411   7.0.7  aca4977fbe8741e1              NaN               NaN   
      150480   7.0.7  aca4977fbe8741e1              NaN               NaN   
10001 214008   6.6.2  09baf2f39bc3dc86  HUAWEI GRA-CL00  e56819f72c9b7860   

                  ...           device_code3 mac1              mac2  \
UID               ...                                                 
10000 307385      ...       deb68f7028682008  NaN               NaN   
      420183      ...       deb68f7028682008  NaN               NaN   
      87507       ...       deb68f7028682008  NaN               NaN   
      216744      ...       deb68f7028682008  NaN               NaN   
      96940       ...       deb68f7028682008  NaN               NaN   
      196361      ...       deb68f7028682008  NaN               NaN   
      168206      ...       deb68f7028682008  NaN               NaN   
      339411      ...       deb68f7028682008  NaN               NaN   
      150480      ...       deb68f7028682008  NaN               NaN   
10001 214008      ...                    NaN  NaN  978e38f2d0431a30   

                           ip1  ip2 wifi geo_code           ip1_sub ip2_sub  \
UID                                                                           
10000 307385  31599eec867ba9c7  NaN  NaN      NaN  ee08d48e73120da4     NaN   
      420183  31599eec867ba9c7  NaN  NaN      NaN  ee08d48e73120da4     NaN   
      87507   31599eec867ba9c7  NaN  NaN      NaN  ee08d48e73120da4     NaN   
      216744  31599eec867ba9c7  NaN  NaN      NaN  ee08d48e73120da4     NaN   
      96940   34c715882de1a508  NaN  NaN      NaN  1e3ea9498c461cbf     NaN   
      196361  34c715882de1a508  NaN  NaN      NaN  1e3ea9498c461cbf     NaN   
      168206  34c715882de1a508  NaN  NaN      NaN  1e3ea9498c461cbf     NaN   
      339411  34c715882de1a508  NaN  NaN      NaN  1e3ea9498c461cbf     NaN   
      150480  34c715882de1a508  NaN  NaN      NaN  1e3ea9498c461cbf     NaN   
10001 214008  2c1958d001a993d9  NaN  NaN     wsgb  bd5e6a16e61e61f7     NaN   

                 timestamp  
UID                         
10000 307385  1.534141e+09  
      420183  1.534141e+09  
      87507   1.534141e+09  
      216744  1.534141e+09  
      96940   1.535257e+09  
      196361  1.535257e+09  
      168206  1.535257e+09  
      339411  1.535257e+09  
      150480  1.535257e+09

In [11]:
tran_train_gb.head(10)

UID  channel  day                 time  trans_amt  \
UID                                                                 
10000 90698   10000      140   26  2018-08-26 12:23:56       5536   
      134052  10000      140   26  2018-08-26 12:24:17       5536   
10001 98298   10001      140    2  2018-08-02 08:31:13        154   
      63298   10001      140    2  2018-08-02 08:31:13       9287   
      42602   10001      102    3  2018-08-03 07:18:17      15186   
      89982   10001      102    4  2018-08-04 22:30:51      15186   
      78949   10001      102    7  2018-08-07 07:30:48      10293   
      126895  10001      102    7  2018-08-07 07:34:35      81648   
      45908   10001      102    7  2018-08-07 16:12:52      13628   
      23578   10001      102    8  2018-08-08 07:49:23      15186   

                      amt_src1          merchant code1 code2  \
UID                                                            
10000 90698   f29829bc82459191  88aa547576f43f85   NaN   NaN   
      134052  f29829bc82459191  88aa547576f43f85   NaN   NaN   
10001 98298   c5fc631370cabc0d  f3237791ad270d6b   NaN   NaN   
      63298   4d7831c6f695ab19  f3237791ad270d6b   NaN   NaN   
      42602   155c9e1c32bd0fa2  a18cb3d84b9ec04c   NaN   NaN   
      89982   155c9e1c32bd0fa2  a18cb3d84b9ec04c   NaN   NaN   
      78949   155c9e1c32bd0fa2  a18cb3d84b9ec04c   NaN   NaN   
      126895  9451ef3c5a0d6807  3f6d3d0f42519ea4   NaN   NaN   
      45908   155c9e1c32bd0fa2  3bd5cf7c40962299   NaN   NaN   
      23578   155c9e1c32bd0fa2  a18cb3d84b9ec04c   NaN   NaN   

                   trans_type1      ...         bal          amt_src2  \
UID                                 ...                                 
10000 90698   c2f2023d279665b2      ...         100  9a8ee16bde15e38a   
      134052  c2f2023d279665b2      ...         100  9a8ee16bde15e38a   
10001 98298   c2f2023d279665b2      ...         100               NaN   
      63298   c2f2023d279665b2      ...         100               NaN   
      42602   6d55c54c8b1056fb      ...       61204  9fefed0a981dcb7a   
      89982   6d55c54c8b1056fb      ...       46125  9fefed0a981dcb7a   
      78949   6d55c54c8b1056fb      ...       22354  9fefed0a981dcb7a   
      126895  6d55c54c8b1056fb      ...         100  8d2ff85b6fd5dc78   
      45908   e0d7b8768da99dd4      ...         100  cf6e3a074407c379   
      23578   6d55c54c8b1056fb      ...       75290  9fefed0a981dcb7a   

                       acc_id2           acc_id3 geo_code trans_type2  \
UID                                                                     
10000 90698                NaN               NaN      NaN       105.0   
      134052               NaN               NaN      NaN       105.0   
10001 98298                NaN               NaN     wsgb       105.0   
      63298                NaN               NaN     wsgb       105.0   
      42602                NaN               NaN     wssu       102.0   
      89982                NaN               NaN     wssp       102.0   
      78949                NaN               NaN     wssp       102.0   
      126895               NaN               NaN     wssp       102.0   
      45908   65407f8f309309a6  4c287b179b08eb62     wssp       102.0   
      23578                NaN               NaN     wssp       102.0   

             market_code market_type           ip1_sub     timestamp  
UID                                                                   
10000 90698          NaN         NaN  1e3ea9498c461cbf  1.535257e+09  
      134052         NaN         NaN  1e3ea9498c461cbf  1.535257e+09  
10001 98298          NaN         NaN  bd5e6a16e61e61f7  1.533170e+09  
      63298          NaN         NaN  bd5e6a16e61e61f7  1.533170e+09  
      42602          NaN         NaN  8e6d45472f232ed4  1.533252e+09  
      89982          NaN         NaN  8f396a6bd85f9ba3  1.533393e+09  
      78949          NaN         NaN  509cdabae18d3c24  1.533598e+09  
      126895         NaN 

In [13]:
tag_train[tag_train['UID'] == 10000]

,UID,Tag
0,10000,1


In [ ]:
op_train_uids = [uid for uid, item in op_train.groupby('UID', as_index=False).groups.items()]
tran_train_uids = [uid for uid, item in tran_train.groupby('UID', as_index=False).groups.items()]
tag_train_uids = [uid for uid, item in tag_train.groupby('UID', as_index=False).groups.items()]
print(len(op_train_uids))
print(len(tran_train_uids))
print(len(tag_train_uids))
# op_train_uids = np.array(op_train_uids)
# tran_train_uids = np.array(tran_train_uids)

In [ ]:
op_tag_uids = find_same_uid(op_train_uids, tag_train_uids)
tran_tag_uids = find_same_uid(tran_train_uids, tag_train_uids)
op_tran_tag_uids = find_same_uid(op_train_uids, tran_train_uids)

In [ ]:
print(len(op_tag_uids))
print(len(tran_tag_uids))
print(len(op_tran_tag_uids))

In [ ]:
op_train_gb.count()

In [ ]:
tran_train_gb.count()

In [ ]:
op_train_gb.head()

In [ ]:
tag_true = tag_train[tag_train['Tag']==1]
tag_false = tag_train[tag_train['Tag']==0]

In [ ]:
len_op_true=[]
len_tran_true=[]
len_op_false=[]
len_tran_false=[]
%timeit
print('start tag_true...')
for idx in tag_true['UID'].values:
    len_op_true.append(operation_len(idx))
    len_tran_true.append(transaction_len(idx))
print('start tag_false...')
for idx in tag_false['UID'].values:
    len_op_false.append(operation_len(idx))
    len_tran_false.append(transaction_len(idx))

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
len_op_true = np.array(len_op_true)
len_tran_true = np.array(len_tran_true)
len_op_false = np.array(len_op_false)
len_tran_false = np.array(len_tran_false)

In [ ]:
print(len_op_true.shape, len_tran_true.shape)
print(len_op_false.shape, len_tran_false.shape)

In [ ]:
plt.plot(len_op_true)

In [ ]:
plt.plot(len_tran_true)

In [ ]:
plt.plot(len_op_false)

In [ ]:
plt.plot(len_tran_false)